In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# download spark3.0.0
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

In [ ]:
# unzip it
!tar xf /content/spark-3.0.0-bin-hadoop3.2.tgz

In [ ]:
# install findspark 
!pip install -q findspark

In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [3]:
# Check the pyspark version
import pyspark
print(pyspark.__version__)

3.0.0


In [ ]:
!git clone https://github.com/henu/bigjson

Cloning into 'bigjson'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 143 (delta 22), reused 39 (delta 15), pack-reused 95
Receiving objects: 100% (143/143), 30.08 KiB | 6.01 MiB/s, done.
Resolving deltas: 100% (72/72), done.


In [4]:
from pyspark import SparkContext
from bigjson import bigjson
sc =SparkContext('local')
spark = SparkSession(sc)
# create spark Context for our data
path = "/content/drive/MyDrive/League of Legends - Patch 10.9 Dataset/matches.json"
#raw_data = sc.textFile(path)
#Data = spark.read.json(path,multiLine=True)   

In [5]:
import json
'''
This function reads the matches.json file and returns a block of it
    INPUTS:
        - start: the start index
        - chunk: the chunk size to read from the file
        - filename: the file path
    OUTPUTS:
        data_now: a list of json objects from the file
'''
def read_data(start,chunk,filename):
  data_now = []
  with open(filename, 'rb') as f:
    data = bigjson.load(f)

    #read data
    for i in range(start, start+chunk):
      data_json = json.dumps(data[i].to_python())
      data_now.append(str(data_json))
    return data_now

In [6]:
Sample = read_data(0,100,path)

In [7]:
rawJson = sc.parallelize(Sample)
rawDF = spark.read.json(rawJson)

In [15]:
from pyspark.sql.types import *
schema1 = StructType([
  StructField('key', StringType(), True),
  StructField('value', IntegerType(), True)
  ])
schema2 = StructType([
  StructField('key', StringType(), True),
  StructField('value1', IntegerType(), True),
  StructField('value2', IntegerType(), True)
  ])
schema3 = StructType([
  StructField('key', StringType(), True),
  StructField('value1', IntegerType(), True),
  StructField('value2', IntegerType(), True),
  StructField('value3', IntegerType(), True)
  ])
schema4 = StructType([
  StructField('key', StringType(), True),
  StructField('value1', StringType(), True),
  StructField('value2', IntegerType(), True),
  StructField('value3', IntegerType(), True),
  StructField('value4', IntegerType(), True)
  ])
ban_df = spark.createDataFrame(spark.sparkContext.emptyRDD(),schema1)
pick_df = spark.createDataFrame(spark.sparkContext.emptyRDD(),schema1)
win_df = spark.createDataFrame(spark.sparkContext.emptyRDD(),schema1)
synergy_df = spark.createDataFrame(spark.sparkContext.emptyRDD(),schema1)
item_pick_df = spark.createDataFrame(spark.sparkContext.emptyRDD(),schema1)
item_win_df = spark.createDataFrame(spark.sparkContext.emptyRDD(),schema1)
item_synergy_df = spark.createDataFrame(spark.sparkContext.emptyRDD(),schema1)
#ban_df.printSchema()
#pick_df.printSchema()
#win_df.printSchema()
#synergy_df.printSchema()
item_pick_df.printSchema()
#item_win_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: integer (nullable = true)



In [17]:
#banrate calculations
for i in range(2):
  for j in range(5):
    ban = rawDF.rdd.map(lambda x: (x['teams'][i]['bans'][j]['championId'],1)).reduceByKey(lambda x,y: x+y).toDF(['key','value'])
    #bandDF = ban.toDF(['key','value'])
    ban_df = ban_df.union(ban)
    count = rawDF.count()
ban_local = ban_df.rdd.reduceByKey(lambda x,y: x+y).toDF(['key','value'])
#ban_g = ban_g.union(ban).reduceByKey(lambda x,y: x+y)

In [16]:
#pickrate and winrate calculations
#item win and pick rates calculations
#item synergies calculations
for k in range(10):
  pick    =  rawDF.rdd.map(lambda x:(x['participants'][k]['championId'],1)).reduceByKey(lambda x,y: x+y).toDF(['key','value'])
  win     =  rawDF.rdd.map(lambda x:(x['participants'][k]['championId'],(x['participants'][k]['stats']['win']&1))).reduceByKey(lambda x,y: x+y).toDF(['key','value'])
  pick_df = pick_df.union(pick)
  win_df  = win_df.union(win)
  for i in range(6):
    item_pick_df_temp = spark.createDataFrame(spark.sparkContext.emptyRDD(),schema2)
    item_win_df_temp = spark.createDataFrame(spark.sparkContext.emptyRDD(),schema4)
    w = rawDF.rdd.map(lambda x:(x['participants'][k]['stats']['item{}'.format(i)]['name'],len(str(x['participants'][k]['stats']['item{}'.format(i)]['from']))-2,len(str(x['participants'][k]['stats']['item{}'.format(i)]['into']))-2)).toDF(['key','value1','value2'])
    a = rawDF.rdd.map(lambda x:(x['participants'][k]['championId'],x['participants'][k]['stats']['item{}'.format(i)]['name'],x['participants'][k]['stats']['win']&1,len(str(x['participants'][k]['stats']['item{}'.format(i)]['from']))-2,len(str(x['participants'][k]['stats']['item{}'.format(i)]['into']))-2)).toDF(['key','value1','value2','value3','value4'])
    item_pick_df_temp = item_pick_df_temp.union(w)
    item_win_df_temp = item_win_df_temp.union(a)
    item_pick_df_temp = item_pick_df_temp.filter((item_pick_df_temp.value1 >0) & (item_pick_df_temp.value2 == 0))
    item_win_df_temp = item_win_df_temp.filter((item_win_df_temp.value3 >0) & (item_win_df_temp.value4 == 0))
    item1 = item_pick_df_temp.rdd.map(lambda x: (x['key'],1)).reduceByKey(lambda x,y: x+y).toDF(['key','value'])
    item2 = item_win_df_temp.rdd.map(lambda x: (x['value1'],x['value2'])).reduceByKey(lambda x,y: x+y).toDF(['key','value'])
    item3 = item_win_df_temp.rdd.map(lambda x: (x['key']+'|'+x['value1'],x['value2'])).reduceByKey(lambda x,y: x+y).toDF(['key','value'])
    item_pick_df =  item_pick_df.union(item1)
    item_win_df  =  item_win_df.union(item2)
    item_synergy_df = item_synergy_df.union(item3)
pick_local  = pick_df.rdd.reduceByKey(lambda x,y: x+y).toDF(['key','value'])
win_local   = win_df.rdd.reduceByKey(lambda x,y: x+y).toDF(['key','value'])
item_pick_local = item_pick_df.rdd.reduceByKey(lambda x,y: x+y).toDF(['key','value'])
item_win_local = item_win_df.rdd.reduceByKey(lambda x,y: x+y).toDF(['key','value'])
item_synergy_local = item_synergy_df.rdd.reduceByKey(lambda x,y: x+y).toDF(['key','value'])

In [18]:
#function to sort champions
def convert(a,b):
  s = sorted([a,b])
  a = s[0]+'|'+s[1]
  return a
#Champion Synergies calculations
synergy_df = spark.createDataFrame(spark.sparkContext.emptyRDD(),schema1)
for i in range(0,5):
  for j in range(i+1,5):
    synergy     =  rawDF.rdd.map(lambda x:(convert(x['participants'][i]['championId'],x['participants'][j]['championId']),x['participants'][i]['stats']['win']&1)).reduceByKey(lambda x,y: x+y).toDF(['key','value'])
    synergy_df  = synergy_df.union(synergy)
for i in range(5,10):
  for j in range(i+1,10):
    synergy     =  rawDF.rdd.map(lambda x:(convert(x['participants'][i]['championId'],x['participants'][j]['championId']),x['participants'][i]['stats']['win']&1)).reduceByKey(lambda x,y: x+y).toDF(['key','value'])
    synergy_df  = synergy_df.union(synergy)
synergy_df = synergy_df.rdd.reduceByKey(lambda x,y: x+y).toDF(['key','value'])


In [33]:
#item suggestion
#'Ezreal''Varus'
from pyspark.sql.functions import col
from pyspark.sql.functions import * 
item_1_sugg = item_synergy_local.filter(col('key').startswith('Ezreal')).sort(desc('value')).head(10)
item_2_sugg = item_synergy_local.filter(col('key').startswith('Varus')).sort(desc('value')).head(10)

In [2]:
!pip install riotwatcher

     |████████████████████████████████| 61kB 3.4MB/s 


In [13]:
from riotwatcher import LolWatcher, ApiError
API_Key = 'RGAPI-6b26cef2-4928-406c-89f8-8307a9f9e0ea'
watcher = LolWatcher(API_Key) 
region = 'euw1'
games = watcher.spectator.featured_games(region)

In [14]:
#match = watcher.match.by_id(region,games['gameList'][0]['gameId'])
match = watcher.match.by_id(region,5003481305)

HTTPError: ignored

In [75]:
import json
game = json.loads(json.dumps(match))

In [1]:
team1_info_level = {}
team1_info_gold = {}
team2_info_level = {}
team2_info_gold = {}
for i in range(5):
  team1_info_level[game['participantIdentities'][i]['player']['summonerName']] = int(game['participants'][i]['stats']['champLevel'])
  team1_info_gold[game['participantIdentities'][i]['player']['summonerName']] = int(game['participants'][i]['stats']['goldEarned'])
for i in range(5,10):
  team2_info_level[game['participantIdentities'][i]['player']['summonerName']] = int(game['participants'][i]['stats']['champLevel'])
  team2_info_gold[game['participantIdentities'][i]['player']['summonerName']] = int(game['participants'][i]['stats']['goldEarned'])


NameError: ignored

In [63]:
match

{'gameCreation': 1609201981375,
 'gameDuration': 640,
 'gameId': 2147507343,
 'gameMode': 'ARAM',
 'gameType': 'MATCHED_GAME',
 'gameVersion': '10.25.350.1724',
 'mapId': 12,
 'participantIdentities': [{'participantId': 1,
   'player': {'accountId': 'AWvkxiD85aG7Fq0a65tpLfKyiyy5Oc2H7rn08Kxirszml3g',
    'currentAccountId': 'AWvkxiD85aG7Fq0a65tpLfKyiyy5Oc2H7rn08Kxirszml3g',
    'currentPlatformId': 'BR1',
    'matchHistoryUri': '/v1/stats/player_history/BR1/201060364',
    'platformId': 'BR1',
    'profileIcon': 4201,
    'summonerId': 'cQVSVLT3XaG8ShTBQ-QRnIGWxzq1a7J7T2-ZOJHgiRLcoQ',
    'summonerName': 'Zoom z'}},
  {'participantId': 2,
   'player': {'accountId': '_Xl2sFB1PpZCq-rYQ3hJga9wm9ch24ots0GRlFPDg7wH8gY',
    'currentAccountId': '_Xl2sFB1PpZCq-rYQ3hJga9wm9ch24ots0GRlFPDg7wH8gY',
    'currentPlatformId': 'BR1',
    'matchHistoryUri': '/v1/stats/player_history/BR1/200055443',
    'platformId': 'BR1',
    'profileIcon': 4835,
    'summonerId': 'yJqq5es7Uvs7WfqIA-_G55vBchoYnVZDQL9